In [1]:
### WWA
# Import Python standard library and IPython packages we need.
import os
import sys
import subprocess
import matplotlib.pyplot as plt
import pandas as pd

from IPython.display import Image

# Ask GRASS GIS where its Python packages are.
grass_call = "grass82"
sys.path.append(
    subprocess.check_output([grass_call, "--config", "python_path"], text=True, shell=True).strip()
)

# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

# Create a GRASS GIS session.
gj.init("D:/GLISA-WWA", "WWA", "WWA_22")
os.chdir(r'D:\GLISA-WWA\Paper_runs\Colorado_springs')

In [2]:
def lu_stats_CS(run, hazard):
    lustats = gs.read_command('r.stats', flags = 'c', input=run+',WWA_NLCD_16@PERMANENT')
    ag_loss = forest_loss = risk_dev = newurb = 0
    for line in lustats.strip().splitlines():
        cat1,cat2,cells = line.split()
        if cat1 == '1':
            if cat2 in ['81','82']:
                ag_loss += int(cells)
            elif cat2 in ['41','42','43']:
                forest_loss += int(cells)
        
    hazstats = gs.read_command('r.stats', flags = 'c', input=run+','+hazard) 
    for line in hazstats.strip().splitlines():
        cat1,cat2,cells = line.split()
        if cat1 == '1' and cat2 == '1':
            risk_dev += int(cells)
            
    urbstats = gs.read_command('r.stats', flags = 'c', input = run+',urban_2016@PERMANENT')
    for line in urbstats.strip().splitlines():
        cat1,cat2,cells = line.split()
        if cat1 == '1' and cat2 != '1':
            newurb += int(cells)
    
    return [ag_loss, forest_loss, risk_dev, newurb]

In [3]:
gs.run_command('r.mask', raster='statsmask@Hindcasting22')

#initial = lu_stats("urban_2016", "Floodplain_GLISA@PERMANENT")

column_names = ['scenario', 'run', 'ag_loss', 'forest_loss', 'risk_dev', 'newurb']
csdf = pd.DataFrame(columns = column_names)

for scenario in ['baseline_run', 'baseline_migration_run','baseline_migration2x_run','baseline_migration4x_run', 
                 'infill_run', 'infill_migration_run', 'infill_migration2x_run','infill_migration4x_run',
                 'infill_nofire_run', 'infill_nofire_migration_run', 'infill_nofire_migration2x_run','infill_nofire_migration4x_run', 
                 'nofire_run', 'nofire_migration_run', 'nofire_migration2x_run', 'nofire_migration4x_run']:
    for i in range(1, 51):
        run = scenario + str(i)
        runstats = lu_stats_CS(run, "wildfire_top_quintile")
        csdf.loc[len(csdf)] = [i, scenario] + runstats
        if i == 1:
            gs.mapcalc("freqmap_" + scenario + " = " + scenario + "1")
        else:
            gs.mapcalc("freqmap_" + scenario + " = freqmap_"+scenario+" + " + scenario + str(i))
csdf.head()


,scenario,run,ag_loss,forest_loss,risk_dev,newurb
0,1,baseline_run,2945,3476,88410,68625
1,2,baseline_run,4067,5744,91388,66203
2,3,baseline_run,3604,4768,93880,65984
3,4,baseline_run,4109,2979,88405,65391
4,5,baseline_run,3766,2657,87700,65381


In [4]:
csdf.to_csv('csdf_migration2.csv')

In [5]:
### GLISA
# Import Python standard library and IPython packages we need.
import os
import sys
import subprocess
import matplotlib.pyplot as plt
import pandas as pd

from IPython.display import Image

# Ask GRASS GIS where its Python packages are.
grass_call = "grass82"
sys.path.append(
    subprocess.check_output([grass_call, "--config", "python_path"], text=True, shell=True).strip()
)

# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

# Create a GRASS GIS session.
gj.init("D:/GLISA-WWA", "GLISA", "GR22")
os.chdir(r'D:\GLISA-WWA\Paper_runs\Grand_rapids')

In [6]:
def lu_stats_GR(run, hazard):
    lustats = gs.read_command('r.stats', flags = 'c', input=run+',NLCD_2016_GLISA@PERMANENT')
    ag_loss = forest_loss = risk_dev = newurb = 0
    for line in lustats.strip().splitlines():
        cat1,cat2,cells = line.split()
        if cat1 == '1':
            if cat2 in ['81','82']:
                ag_loss += int(cells)
            elif cat2 in ['41','42','43']:
                forest_loss += int(cells)
        
    hazstats = gs.read_command('r.stats', flags = 'c', input=run+','+hazard) 
    for line in hazstats.strip().splitlines():
        cat1,cat2,cells = line.split()
        if cat1 == '1' and cat2 == '1':
            risk_dev += int(cells)
            
    urbstats = gs.read_command('r.stats', flags = 'c', input = run+',urban_2016@PERMANENT')
    for line in urbstats.strip().splitlines():
        cat1,cat2,cells = line.split()
        if cat1 == '1' and cat2 != '1':
            newurb += int(cells)
    
    return [ag_loss, forest_loss, risk_dev, newurb]

In [7]:
gs.run_command('r.mask', raster='statsmask@Hindcast22')

#initial = lu_stats("urban_2016", "Floodplain_GLISA@PERMANENT")

column_names = ['scenario', 'run', 'ag_loss', 'forest_loss', 'risk_dev', 'newurb']
grdf = pd.DataFrame(columns = column_names)

for scenario in ['baseline_run', 'baseline_migration_run', 'baseline_migration2x_run', 'baseline_migration4x_run',
                 'infill_run', 'infill_migration_run', 'infill_migration2x_run', 'infill_migration4x_run', 
                 'infill_noflood_run', 'infill_noflood_migration_run', 'infill_noflood_migration2x_run', 'infill_noflood_migration4x_run', 
                 'noflood_run', 'noflood_migration_run', 'noflood_migration2x_run', 'noflood_migration4x_run']:
    for i in range(1, 51):
        run = scenario + str(i)
        runstats = lu_stats_GR(run, "Floodplain_GLISA")
        grdf.loc[len(grdf)] = [i, scenario] + runstats
        if i == 1:
            gs.mapcalc("freqmap_" + scenario + " = " + scenario + "1")
        else:
            gs.mapcalc("freqmap_" + scenario + " = freqmap_"+scenario+" + " + scenario + str(i))
grdf.head()


,scenario,run,ag_loss,forest_loss,risk_dev,newurb
0,1,baseline_run,73241,34008,90291,118615
1,2,baseline_run,73901,33538,90219,118557
2,3,baseline_run,73735,34078,90263,118600
3,4,baseline_run,75158,33176,90220,118596
4,5,baseline_run,75648,32204,90491,118562


In [8]:
grdf.to_csv('grdf_migration2.csv')